[Link](https://www.youtube.com/watch?v=KdoGekqz2hg)

In [12]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from binance import Client
import pandas as pd

#Config
import sys
sys.path.insert(0,"/Users/marvinottersberg/Documents/GitHub/sentiment/")
from config import ConfigBinance
conf = ConfigBinance().getKeys()

client = Client(conf[0],conf[1])

plt.style.use("ggplot")
print(plt.style.available)

asset = "BTCUSDT"



['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [13]:
def getminutedata(symbol,interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame = frame.iloc[:,:6]
    frame.columns= ["Time","Open","High","Low","Close","Volume"]
    frame = frame.set_index("Time")
    frame.index = pd.to_datetime(frame.index,unit = "ms")
    frame = frame.astype(float)
    return frame


In [14]:
df = getminutedata(asset, "1m","120m") #data every 1 minute for total lookback of 120 minutes

df


,Open,High,Low,Close,Volume
Time,,,,,
2022-07-14 05:00:00,20083.01,20105.94,20077.09,20102.63,162.45928
2022-07-14 05:01:00,20100.98,20118.53,20080.10,20082.64,148.31744
2022-07-14 05:02:00,20082.64,20087.46,20066.45,20073.18,91.93017
2022-07-14 05:03:00,20072.33,20094.40,20064.17,20090.25,146.82108
2022-07-14 05:04:00,20090.25,20104.95,20080.60,20099.52,58.56203
...,...,...,...,...,...
2022-07-14 06:55:00,20018.27,20027.15,20012.26,20020.66,64.79446
2022-07-14 06:56:00,20020.66,20032.00,20017.05,20020.59,72.22679
2022-07-14 06:57:00,20022.15,20032.73,20009.49,20028.67,110.66727


In [15]:
def animate(i):
    data = getminutedata(asset,"1m","120m")
    #plt.figure(figsize=((10,8)))
    plt.cla()
    plt.plot(data.index,data.Close)
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.title(asset)
    plt.gcf().autofmt_xdate() #adjust dates 
    plt.tight_layout()
    

In [16]:
def getPlot():
    ani = FuncAnimation(plt.gcf(),animate,1000) #updating every second
    plt.figure(figsize=((10,8)))
    plt.tight_layout()
    plt.show()
    return ani

anim = getPlot()

<Figure size 432x288 with 0 Axes>

<Figure size 720x576 with 0 Axes>

## Read Sentiment from Json and Visualise

In [17]:
import os
import pandas as pd

In [18]:

def getSentiment():
    json_path = "sentiment/Json/12-07-2022/btc.json"

    df = pd.read_json(json_path,orient="index")
    df = df.filter(items=["Time","Sentiment Score"])
    #df_unique = df["Time"].unique()
    df["Time"] = pd.to_datetime(df["Time"])
    df_mean= df.resample("1T",on="Time").mean()
    newdf = pd.DataFrame(df_mean)
    #print(df_mean)
    #print(newdf.to_string())

